In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import Embedding

conf = SparkConf().setAppName('ctrModel').setMaster('local[4]')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
    # Change to your own filepath

rawSampleDataPath = "ratings.csv"
embLength = 10
samples = Embedding.processItemSequence(spark, rawSampleDataPath)
model = Embedding.trainItem2vec(spark, samples, embLength,
                          embOutputPath="emb/item2vecEmb.csv", saveToRedis=False,
                          redisKeyPrefix="i2vEmb")
Embedding.graphEmb(samples, spark, embLength, embOutputFilename="itemGraphEmb.csv",
             saveToRedis=True, redisKeyPrefix="graphEmb")
Embedding.generateUserEmb(spark, rawSampleDataPath, model, embLength,
                    embOutputPath="emb/userEmb.csv", saveToRedis=False,
                    redisKeyPrefix="uEmb")

In [ ]:
'/'.join(embOutputPath.split('/')[:-1])